In [1]:
import xarray as xr
import dask
import numpy as np
import os
import time
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1','--ntasks-per-node=24'],memory='120GB',interface='ib0') 
print(cluster.job_script()) 

cluster.scale(240) 



#!/usr/bin/env bash

#SBATCH -J make_profiles
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=112G
#SBATCH -t 00:30:00
#SBATCH --constraint=HSW24
#SBATCH --exclusive
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=24
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://172.30.100.1:44930 --nthreads 0 --nprocs 28 --memory-limit 4.29GB --name make_profiles--${JOB_ID}-- --death-timeout 60 --local-directory $SCRATCHDIR/daskjobqueue --interface ib0



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster)


In [4]:
client

Client Scheduler: tcp://172.30.100.1:44930 Dashboard: http://172.30.100.1:33945/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)


In [6]:
import zarr
def zarr_day(date):
    print('zarrifying T ',str(date))
    year=date[0:4]
    month=date[4:6]
    day=date[6:]

    if month in ['07','08','09','10']:
        CASE='BLB002'
    elif month in ['12','01','02','03','04','05','06']:
        CASE='BLB002X'
    elif month == '11':
        if day in ['01','02','03','04','05','06','07']:
            CASE='BLB002'
        else:
            CASE='BLB002X'
    compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)                   

    datadir='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-'+str(CASE)+'-S/'
    files=sorted(glob.glob(datadir+'*/eNATL60-'+str(CASE)+'_1h_*_gridT_'+year+month+day+'-'+year+month+day+'.nc'))
    data=xr.open_mfdataset(files[0],chunks={'time_counter':1,'y':1000,'x':1000,'deptht':1})['votemper']

    ds=data.to_dataset(name='votemper')

    encoding = {vname: {'compressor': compressor} for vname in ds.variables}
    ds.to_zarr(store='/scratch/cnt0024/hmg2840/albert7a/eNATL60/scale/zarr_temp3D_20100101', encoding=encoding)

    print('zarr done')





In [8]:
%time zarr_day('20100101')

zarrifying T  20100101
zarr done
CPU times: user 15min, sys: 1min 21s, total: 16min 21s
Wall time: 18min 46s


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")
